In [1]:
# import Libraries
import pandas as pd
import csv
import numpy as np

### Cleaning the items table

In [2]:
# cleaning the item table because of a "UnicodeDecodeError" while trying to import into SQL database
# Read dataset
with open('unclean_data\items.csv', 'r', encoding='latin1') as file:
    reader = csv.reader(file)
    dataset = list(reader)
    
# Identify and replace invalid characters
cleaned_dataset = []
for row in dataset:
    cleaned_row = []
    for item in row:
        cleaned_item = ''.join(char for char in item if ord(char) < 128)  # Remove non-ASCII characters
        cleaned_row.append(cleaned_item)
    cleaned_dataset.append(cleaned_row)

In [3]:
# Extract the first row as column names
column_names = cleaned_dataset[0]

# Remove the first row from the dataset
data_without_header = cleaned_dataset[1:]

# Create DataFrame with the first row as header
item_df = pd.DataFrame(data_without_header, columns=column_names)

item_df.head()

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.5,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans


In [4]:
# Remove "a." prefix
item_df['desc'] = item_df['desc'].str.replace('a. ', '')

# Split column by "-"
item_df[['Category', 'Sub-Category']] = item_df['desc'].str.split(r'\s*-\s*|\s+', n=1, expand=True)

# Delete description column from the dataframe
item_df.drop(columns=['desc'], inplace=True)

In [5]:
# Convert 'unit_price' to numeric data type
item_df['unit_price'] = pd.to_numeric(item_df['unit_price'])

# Some Feature Engineering
# Creating additional column for cost
deduct_values = np.random.choice([2, 2.75, 2.5, 3, 3.2, 3.75, 4], size=len(item_df))

item_df['unit_cost'] = item_df['unit_price'] - deduct_values

In [6]:
# rearrange the columns
item_df = item_df[['item_key','item_name',	'Category', 'Sub-Category', 'unit_price', 'unit_cost', 'man_country',	'supplier',	'unit']]

item_df.head(15)

,item_key,item_name,Category,Sub-Category,unit_price,unit_cost,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,Beverage,Soda,11.50,8.75,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,Beverage,Soda,6.75,4.25,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,Beverage,Soda,6.75,4.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,Beverage,Soda,6.75,3.00,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,Beverage,Soda,6.75,2.75,Finland,HARDFORD AB,cans
5,I00006,Coke Classic 12 oz cans,Beverage,Soda,16.25,13.25,Lithuania,BIGSO AB,cans
6,I00007,Coke Classic 12 oz cans,Beverage,Soda,6.75,2.75,India,Indo Count Industries Ltd,cans
7,I00008,Coke Zero Sugar 12 oz cans,Beverage,Soda,16.25,13.25,India,Indo Count Industries Ltd,cans
8,I00009,Diet Coke - 12 oz cans,Beverage,Soda,16.25,13.25,Netherlands,Bolsius Boxmeer,cans
9,I00010,Diet Coke - 12 oz cans,Beverage,Soda,6.75,2.75,Lithuania,BIGSO AB,cans


In [23]:
# export dataframe to a csv file
item_df.to_csv('clean_data\items_cleaned.csv',index=False )

### Cleaning the population table

In [7]:
# Add the population table
# read population csv into df and use second row as header
population_table = pd.read_excel("unclean_data\population.xlsx", header=1)

In [8]:
# Create dataframe
population_df = pd.DataFrame(population_table)
population_df.head()

,Geographic Area,"April 1, 2020 Estimates Base",2020,2021,2022,2023
0,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,331464948.0,331526933.0,332048977.0,333271411.0,334914895.0
2,Northeast,57614141.0,57430477.0,57243423.0,57026847.0,56983517.0
3,Midwest,68987296.0,68969794.0,68850246.0,68783028.0,68909283.0
4,South,126268529.0,126465281.0,127353282.0,128702030.0,130125290.0


In [9]:
# remove first row (contains null values) and 2nd to 5th which contains territories
# which will be used in another column 
population_df.drop(population_df.index[:6], inplace=True)

population_df.head()

,Geographic Area,"April 1, 2020 Estimates Base",2020,2021,2022,2023
6,.Alabama,5024294.0,5031864.0,5050380.0,5073903.0,5108468.0
7,.Alaska,733374.0,732964.0,734923.0,733276.0,733406.0
8,.Arizona,7157902.0,7186683.0,7272487.0,7365684.0,7431344.0
9,.Arkansas,3011490.0,3014348.0,3028443.0,3046404.0,3067732.0
10,.California,39538212.0,39503200.0,39145060.0,39040616.0,38965193.0


In [10]:
# drop further null values from the footnotes
population_df = population_df.drop(population_df.tail(8).index)


# remove leading "." character from values in the geographic area column
population_df['Geographic Area'] = population_df['Geographic Area'].str.lstrip('.')

# Drop second column. Not going to be used in dataset
population_df.drop(columns=['April 1, 2020 Estimates Base'], inplace=True)

# rename "Geographic Area" to "Division" to match dataset
population_df.rename(columns={'Geographic Area':'Division'}, inplace=True)


In [11]:
population_df.head()

,Division,2020,2021,2022,2023
6,Alabama,5031864.0,5050380.0,5073903.0,5108468.0
7,Alaska,732964.0,734923.0,733276.0,733406.0
8,Arizona,7186683.0,7272487.0,7365684.0,7431344.0
9,Arkansas,3014348.0,3028443.0,3046404.0,3067732.0
10,California,39503200.0,39145060.0,39040616.0,38965193.0


In [77]:
population_df.to_csv('clean_data/population.csv')